In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama

from pydantic import BaseModel, Field

# structured output
class IsProblemDecisionRoute(BaseModel):
    is_problem: bool = Field(..., description="Reddit Post is relevant to problem seeking, business ideas or problem advices/questions")

# prompt
system_prompt = """
    % ROLE:
    You are an AI assistant that determines if a Reddit post is problem related.
    
    % GOAL:
    Your goal is to determine if the post is seeking a solution to a problem, implementing a new business idea that addresses a problem or asking for advice/questions.
    Use the query '{query}' to determine if the post is also related to the query.
    Follow the instructions below.

    % INSTRUCTIONS:
    - Carefully read the provided post context.
    - If the query is provided, determine if the post is related to the query first. Then, determine if the post is seeking a solution to a problem, implementing a new business idea that addresses a problem or asking for advice/questions in the context of the query. Your response if YES/TRUE.
    - If the query is not provided, determine if the post is seeking a solution to a problem, implementing a new business idea that addresses a problem or asking for advice/questions in general. Your response if YES/TRUE.
    - Otherwise, your response if NO/FALSE.
    - Output format is JSON object.

    % CONTEXT:
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{context}"),
    ]
)

# llm
llm = ChatOllama(model="llama3.2:3b")
structured_llm = llm.with_structured_output(IsProblemDecisionRoute, method="json_schema")

# chain
chain = prompt | structured_llm

In [5]:
#invoke
query = "lead enrichment"
context = "Am currently using smartreach to build lead list, run cold email and make cold calls. The tool offers decent deliverability but lacks data enrichment capabilities. I heard a lot of influencers talking about Clay, but is it worth the cost? Are there other tools out there that could help me enrich at a much lower cost? The founder of Findlymail also confirmed that Clay offers just 2% more emails than them, so is it worth?"
output = chain.invoke({
    'query': query,
    'context': context
})

print(type(output))
print(output)

<class '__main__.IsProblemDecisionRoute'>
is_problem=True
